In [ ]:
%load_ext autoreload
%autoreload
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from BiotSolver import solve_biot
import sympy as sym
from PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, compute_order)

resolutions = [10, 20, 40,]   # resolution

expr_degree = 4
T = 0.2           # final time
num_steps = 10    # number of time steps
dt = T / num_steps

material_parameter = dict()
material_parameter["c"] = 1
material_parameter["K"] = 1e-10
material_parameter["lmbda"] = 16 
material_parameter["mu"] = 0.33
material_parameter["alpha"] = 1

boundary_all = CompiledSubDomain("on_boundary")
boundary_right = CompiledSubDomain("on_boundary && near(x[0], 1)")
boundary_left = CompiledSubDomain("on_boundary && near(x[0], 0)")
boundary_top = CompiledSubDomain("on_boundary && near(x[1], 1)")
boundary_bottom = CompiledSubDomain("on_boundary && near(x[1], 0)")

In [ ]:
from sympy import symbols, pi
from sympy import derive_by_array, tensorcontraction

c, mu, lmbda, alpha, t, K = sym.symbols('c, mu, lmbda, alpha, t, K')
K = material_parameter["K"]

x,y,z = sym.symbols("x, y, z")

ux = (sym.sin(1*pi*x) + sym.sin(1*pi*y))
uy = (sym.cos(1*pi*x) + sym.sin(1*pi*y))
u = sym.Matrix([ux, uy, 0])*t
p = (sym.cos(1*pi*x) + sym.sin(1*pi*y))*t

def row_wise_div(tensor):
    return sym.Matrix(tensorcontraction(derive_by_array( tensor, [x,y,z]), (0,2)))
    
def row_wise_grad(u):
    return u.jacobian([x,y,z])

def eps(u):
    return 0.5*(row_wise_grad(u) + sym.transpose(row_wise_grad(u)))

def div(u):
    return sym.trace(u.jacobian([x,y,z]))

def grad(p):
    return sym.Matrix([sym.diff(p,x), sym.diff(p,y), sym.diff(p,z)])
 
f = - row_wise_div(2*mu*eps(u) + lmbda*div(u)*sym.eye(3)) + grad(p)
g = c*sym.diff(p, t) + alpha*sym.diff(div(u), t) - div(K*grad(p))
f.simplify()
g.simplify()

p_N_right = - sym.diff( p, x).subs(x, 1)
p_N_left = sym.diff(p, x).subs(x, 0)
p_N_bottom = sym.diff(p, y).subs(y, 0)
p_N_top = - sym.diff(p, y).subs(y, 1)


fx, fy = f[0], f[1]
ux, uy = u[0], u[1]

In [ ]:
xcpp, ycpp = sym.symbols('x[0], x[1]')

variables_scalar = [g, p, p_N_right, p_N_left, p_N_bottom, p_N_top]
variables_scalar_cpp  = [sym.printing.ccode(v.subs(x, xcpp).subs(y, ycpp)) for v in variables_scalar]
variables_scalar_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_scalar_cpp]
g, p_e, p_N_right, p_N_left, p_N_bottom, p_N_top = variables_scalar_expr


variables_vector = [f, u]
variables_vector_cpp  = [(sym.printing.ccode(v[0].subs(x, xcpp).subs(y, ycpp)),
                          sym.printing.ccode(v[1].subs(x, xcpp).subs(y, ycpp)))
                          for v in variables_vector]
variables_vector_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_vector_cpp]
f, u_e = variables_vector_expr

In [ ]:
def biot_convergence(resolutions, boundaries,
                     boundary_conditions_p, boundary_conditions_u):
    L2_errors_u = []
    L2_errors_p = []
    h = []
    for N in resolutions:
        p_e.t = 0
        mesh = UnitSquareMesh(N, N)
        h.append(mesh.hmax())

        boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
        
        for b, marker_id in boundaries.items():
            b.mark(boundary_marker, marker_id)
            
        solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                         boundary_marker, boundary_conditions_p,
                         boundary_marker, boundary_conditions_u,
                         #p_initial=p_e, u_initial=u_e,
                         theta=0.5)

        solution = [s.copy() for s in solution]
        J_u = 0
        J_p = 0
        for i,up in enumerate(solution):
            u,pT, p = up.split()
            p_e.t = (i + 1)*dt
            u_e.t = (i + 1)*dt
            J_p += errornorm(p_e, p, "L2")
            J_u += errornorm(u_e, u, "L2")

        L2_errors_u.append(J_u/num_steps)
        L2_errors_p.append(J_p/num_steps)

    return L2_errors_u, L2_errors_p, h, solution    


In [ ]:
# Dirichlet boundary conditions
boundaries = {boundary_all:1}
boundary_conditions_p = {1:{"Dirichlet":p_e}}
boundary_conditions_u = {1:{"Dirichlet":u_e}}
res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p, boundary_conditions_u)
L2_errors_u, L2_errors_p, h, solution = res  

print(compute_order(L2_errors_u, h))
print(compute_order(L2_errors_p, h))

In [ ]:
# Neumann boundary conditions
boundaries = {boundary_top: 1,
              boundary_bottom: 1,
              boundary_left: 2,
              boundary_right: 3}

boundary_conditions_p = {1:{"Dirichlet": p_e},
                         2:{"Neumann": p_N_left},
                         3:{"Neumann": p_N_right},
                         }

boundary_conditions_u = {1:{"Dirichlet":u_e},
                         2:{"Dirichlet":u_e},
                         3:{"Dirichlet":u_e}}

res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p, boundary_conditions_u)
L2_errors_u_neum, L2_errors_p_neum, h, solution = res  

print(compute_order(L2_errors_u_neum, h))
print(compute_order(L2_errors_p_neum, h))

In [ ]:
# Robin boundary conditions
boundaries = {boundary_top: 1,
              boundary_bottom: 1,
              boundary_left: 2,
              boundary_right: 3}

beta = 0.9
boundary_conditions_p = {1:{"Dirichlet":p_e},
                        2:{"Robin": (beta, beta*p_e + p_N_left) },
                        3:{"Robin": (beta, beta*p_e + p_N_right) },
                        }

boundary_conditions_u = {1:{"Dirichlet":u_e},
                         2:{"Dirichlet":u_e},
                         3:{"Dirichlet":u_e}}

res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p, boundary_conditions_u)

L2_errors_u_rob, L2_errors_p_rob, h, solution = res  

print(compute_order(L2_errors_u_rob, h))
print(compute_order(L2_errors_p_rob, h))


In [ ]:
np.array(L2_errors_p_neum) - np.array(L2_errors_p_rob)

In [ ]:
N = resolutions[-1]
mesh = UnitSquareMesh(N, N)

i = 0
u,p_T, p = solution[i].split()
plt.figure(figsize=(10,10))
c = plot(p)
plt.colorbar(c)
plt.figure(figsize=(10,10))
p_e.t = (i + 1)*dt
c = plot(p_e, mesh=mesh)
plt.colorbar(c)
plt.figure(figsize=(10,10))
c = plot(p - p_e, mesh=mesh)
plt.colorbar(c)

In [ ]:
x_coords = np.linspace(0,1,100)
points = [Point(x,0) for x in x_coords]
cs = extract_cross_section([s.split()[2] for s in solution], points)
plt.plot(x_coords, cs[0,:])